In [1]:
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from datetime import timedelta

In [2]:
def make_training_set(n_shifts):
    filepath_train = r'../datasets/ais_train.csv'
    filepath_test = r'../datasets/ais_test.csv'

    # Load AIS historical data
    train = pd.read_csv(filepath_train, sep ='|')  # Replace with your dataset
    test = pd.read_csv(filepath_test, sep = ',')

    # Preprocessing
    train['time'] = pd.to_datetime(train['time'])
    train.sort_values(by=['vesselId', 'time'], inplace=True)
    train['isMoored'] = train['navstat']== 5
    # Why are we using the complement?
    train = train[~train['isMoored']]

    test['time'] = pd.to_datetime(test['time'])
    test.sort_values(by=['vesselId', 'time'], inplace=True)

    # Feature Engineering
    train['prev_lat'] = train.groupby('vesselId')['latitude'].shift(n_shifts)
    train['prev_lon'] = train.groupby('vesselId')['longitude'].shift(n_shifts)
    train['prev_speed'] = train.groupby('vesselId')['sog'].shift(n_shifts)
    train['prev_course'] = (train.groupby('vesselId')['cog'].shift(n_shifts) / 180) - 1        # normalized
    train['prev_rotation'] = train.groupby('vesselId')['rot'].shift(n_shifts) 
    train['prev_heading'] = (train.groupby('vesselId')['heading'].shift(n_shifts)/ 180) - 1 
    train['hour'] = train['time'].dt.hour
    # Could change this to is_weekend
    train['day_of_week'] = train['time'].dt.dayofweek
    # Adding timedelta as a feature
    train['time_diff'] = train['time'].diff(n_shifts)
    train['time_diff_seconds'] = train['time_diff'].dt.total_seconds()

    # --------------------------------- prev_rot-related stuff
    # Replace special values with NaN
    train['prev_rotation'] = train['prev_rotation'].replace({127: np.nan, -127: np.nan, -128: np.nan})
    train['prev_speed'] = train['prev_speed'].replace({102.3: np.nan})
    train['prev_course'] = train['prev_course'].replace({360: np.nan})
    train['prev_heading'] = train['prev_heading'].replace({511: np.nan})
    train.dropna(inplace=True)

    # Create binary columns for turn direction and magnitude
    train['turn_direction'] = np.where(train['prev_rotation'] > 0, 1, 0)
    train['turn_magnitude'] = train['prev_rotation'].abs()

    # # Fill missing values (optional, using forward fill) Uses most recent non-null value from the row above.
    # train['prev_rotation'].fillna(method='ffill', inplace=True)

    # Drop rows with missing values
    train.dropna(inplace=True)

    print(f"Length of dataset after preprocessing: {len(train)}")
    return train
    

In [3]:
# train1 = make_training_set(1)
# train2 = make_training_set(2)
train3 = make_training_set(3)
train4 = make_training_set(4)
train5 = make_training_set(5)
train6 = make_training_set(6)
train7 = make_training_set(7)
train8 = make_training_set(8)
train9 = make_training_set(9)
train10 = make_training_set(10)
train11 = make_training_set(11)
train12 = make_training_set(12)

train = pd.concat([train3, train4, train5, train6, train7, train8, train9, train10, train11, train12], ignore_index=True)

Length of dataset after preprocessing: 891725
Length of dataset after preprocessing: 891056
Length of dataset after preprocessing: 890379
Length of dataset after preprocessing: 889717
Length of dataset after preprocessing: 889046
Length of dataset after preprocessing: 888373
Length of dataset after preprocessing: 887700
Length of dataset after preprocessing: 887031
Length of dataset after preprocessing: 886352
Length of dataset after preprocessing: 885678


In [4]:
feats_to_include = ['prev_lat', 'prev_lon', 'prev_speed', 'prev_course','prev_rotation', 'prev_heading', 'time_diff_seconds']
X = train[feats_to_include]
y_lat = train['latitude']
y_lon = train['longitude']

X_lat_train, X_lat_val, y_lat_train, y_lat_val = train_test_split(X, y_lat, test_size=0.1, random_state=42)
X_lon_train, X_lon_val, y_lon_train, y_lon_val = train_test_split(X, y_lon, test_size=0.1, random_state=42)

In [5]:
# Train the model
model_lat = RandomForestRegressor(n_estimators=10, verbose=3, random_state=42, warm_start=True)
model_lat.fit(X_lat_train.values, y_lat_train.values)

model_lon = RandomForestRegressor(n_estimators=10, verbose=3, random_state=42, warm_start=True)
model_lon.fit(X_lon_train.values, y_lon_train.values)

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


RandomForestRegressor(n_estimators=10, random_state=42, verbose=3,
                      warm_start=True)

In [6]:
model_lat.set_params(n_estimators=20)
model_lon.set_params(n_estimators=20)
model_lat.fit(X_lat_train.values, y_lat_train.values)
model_lon.fit(X_lon_train.values, y_lon_train.values)

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


RandomForestRegressor(n_estimators=20, random_state=42, verbose=3,
                      warm_start=True)

In [7]:
model_lat.set_params(n_estimators=30)
model_lon.set_params(n_estimators=30)
model_lat.fit(X_lat_train.values, y_lat_train.values)
model_lon.fit(X_lon_train.values, y_lon_train.values)

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


MemoryError: could not allocate 1073741824 bytes

In [ ]:
model_lat.set_params(n_estimators=40)
model_lon.set_params(n_estimators=40)
model_lat.fit(X_lat_train.values, y_lat_train.values)
model_lon.fit(X_lon_train.values, y_lon_train.values)

In [ ]:
model_lat.set_params(n_estimators=50)
model_lon.set_params(n_estimators=50)
model_lat.fit(X_lat_train.values, y_lat_train.values)
model_lon.fit(X_lon_train.values, y_lon_train.values)

In [ ]:
model_lat.set_params(n_estimators=60)
model_lon.set_params(n_estimators=60)
model_lat.fit(X_lat_train.values, y_lat_train.values)
model_lon.fit(X_lon_train.values, y_lon_train.values)

In [8]:
# Make predictions on the validation set
y_lat_pred_val = model_lat.predict(X_lat_val)
y_lon_pred_val = model_lon.predict(X_lon_val)

# Evaluate performance on the validation set
mae_lat = mean_absolute_error(y_lat_val, y_lat_pred_val)
mae_lon = mean_absolute_error(y_lon_val, y_lon_pred_val)

print(f'Mean Absolute Error for Latitude: {mae_lat}')
print(f'Mean Absolute Error for Longitude: {mae_lon}')

c:\Users\mariu\OneDrive\Desktop\moderne_maskinlæring\venv_tdt4173\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
c:\Users\mariu\OneDrive\Desktop\moderne_maskinlæring\venv_tdt4173\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


Mean Absolute Error for Latitude: 0.053262656235405985
Mean Absolute Error for Longitude: 0.08617362674577077


In [ ]:

filepath_train = r'../datasets/ais_train.csv'

# Load AIS historical data
training_data = pd.read_csv(filepath_train, sep ='|')  # Replace with your dataset

# Predict future positions
def predict_future_position(id, vessel_id, time):
    # Fetch the latest known position of the vessel
    latest_data = training_data[training_data['vesselId'] == vessel_id].sort_values(by='time').iloc[-1]

    new_data = {
        'prev_lat': latest_data['latitude'],
        'prev_lon': latest_data['longitude'],
        'prev_speed': latest_data['sog'],
        'prev_course': (latest_data['cog'] / 180) - 1,
        'prev_rotation': latest_data['rot'],
        'turn_info_available': 1 if latest_data['rot'] != -128 else 0,
        'prev_heading' : (latest_data['heading'] / 180) - 1,

        # Convert the times to a datetime_obj
        'time_diff_seconds' : (datetime.strptime(time, '%Y-%m-%d %H:%M:%S') - datetime.strptime(latest_data['time'], '%Y-%m-%d %H:%M:%S')).total_seconds(),
    }
    # why is it done with all this list stuff?
    return id, model_lat.predict([list(new_data.values())])[0], model_lon.predict([list(new_data.values())])[0]
    # return id, model_lat.predict([list(new_data.values())]), model_lon.predict([list(new_data.values())]), new_data['time_diff_seconds'], latest_data['time']

# ['prev_lat', 'prev_lon', 'prev_speed', 'prev_course','prev_rotation', 'turn_info_available', 'prev_heading', 'time_diff_seconds']

# Open the test file for reading and the prediction file for writing
with open('../datasets/ais_test.csv', 'r') as f_test, open('../predictions/predictions_2.csv', 'w') as f_pred:
    f_pred.write("ID,longitude_predicted,latitude_predicted\n")
    for line in tqdm(f_test.readlines()[1:]):
        id, vesselID, time, scaling_factor = line.split(',')
        id, pred_lat, pred_lon = predict_future_position(id, vesselID, time)
        f_pred.write(f"{id},{pred_lon},{pred_lat}\n")